<a href="https://colab.research.google.com/github/vidsanchez/RetellAi-Utils/blob/main/clone_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clone Retell AI Agent**

In [ ]:
# @title Get the variables (Source API key, Source Agent ID and Destination API key)
apikey_source = input("Source API key: ")
agent_id_source = input("Source Agent ID: ")
apikey_destination = input("Destination API key: ")

In [7]:
# @title clone_agent function definition
import requests
import json
import sys

def clone_agent(apikey_origen, agent_id, apikey_destino):
    # Step 1. Get the agent configuration from the source agent_id
    agent_url = f'https://api.retellai.com/get-agent/{agent_id}'
    headers_origen = {'Authorization': f'Bearer {apikey_origen}'}


    response = requests.get(agent_url, headers=headers_origen)
    if response.status_code != 200:
        print(f"Error getting agent: {response.status_code}, {response.text}")
        return
    agent_data = response.json()
    print(f"Agent retrieved: {agent_data['agent_id']}")

    # Extract the llm_id from the agent data
    llm_id = None
    if 'llm_id' in agent_data:
        llm_id = agent_data['llm_id']
    elif 'response_engine' in agent_data and 'llm_id' in agent_data['response_engine']:
        llm_id = agent_data['response_engine']['llm_id']
    else:
        print("llm_id not found in agent data")
        return

    # Step 2: Get the llm configuration
    llm_url = f'https://api.retellai.com/get-retell-llm/{llm_id}'
    response = requests.get(llm_url, headers=headers_origen)
    if response.status_code != 200:
        print(f"Error getting LLM: {response.status_code}, {response.text}")
        return
    llm_data = response.json()
    print(f"LLM retrieved: {llm_id}")

    # Step 3: Create a new llm in the destination account
    llm_fields = [
        "model",
        "temperature",
        "use_structured_output",
        "general_prompt",
        "general_tools",
        "states",
        "starting_state",
        "first_message",
        "inbound_call_webhook",
        "knowledge_base_ids"
    ]
    llm_create_data = {}
    for field in llm_fields:
        if field in llm_data:
            llm_create_data[field] = llm_data[field]

    llm_create_url = 'https://api.retellai.com/create-retell-llm'
    headers_destino = {'Authorization': f'Bearer {apikey_destino}', 'Content-Type': 'application/json'}
    response = requests.post(llm_create_url, headers=headers_destino, json=llm_create_data)
    if response.status_code != 201:
        print(f"Error creating LLM: {response.status_code}, {response.text}")
        return
    new_llm_data = response.json()
    new_llm_id = new_llm_data.get('llm_id')
    if not new_llm_id:
        print("Failed to retrieve new LLM ID")
        return
    print(f"LLM created: {new_llm_id}")

    # Step 4: Create a new agent in the destination account
    agent_fields = [
        "agent_name",
        "voice_id",
        "voice_model",
        "voice_fallback_ids",
        "voice_temperature",
        "voice_speed",
        "volume",
        "responsiveness",
        "interruption_sensitivity",
        "enable_backchannel",
        "backchannel_frequency",
        "backchannel_words",
        "reminder_trigger_ms",
        "reminder_max_count",
        "ambient_sound",
        "ambient_sound_volume",
        "language",
        "webhook_url",
        "boosted_keywords",
        "enable_transcription_formatting",
        "opt_out_sensitive_data_storage",
        "pronunciation_dictionary",
        "normalize_for_speech",
        "end_call_after_silence_ms",
        "max_call_duration_ms",
        "enable_voicemail_detection",
        "voicemail_message",
        "voicemail_detection_timeout_ms",
        "post_call_analysis_data"
    ]
    agent_create_data = {"response_engine": {"type": "retell-llm", "llm_id": new_llm_id}}
    for field in agent_fields:
        if field in agent_data:
            agent_create_data[field] = agent_data[field]

    agent_create_url = 'https://api.retellai.com/create-agent'
    response = requests.post(agent_create_url, headers=headers_destino, json=agent_create_data)
    if response.status_code != 201:
        print(f"Error creating agent: {response.status_code}, {response.text}")
        return
    new_agent_data = response.json()

    new_agent_id = new_agent_data.get('agent_id')
    if not new_agent_id:
        print("Failed to retrieve new agent ID")
        return

    print(f"New agent created with ID: {new_agent_id}")


In [ ]:
# @title Clone Agent
clone_agent(apikey_source, agent_id_source, apikey_destination)